In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pyspark

In [50]:
sc = pyspark.SparkContext("local", "App Name")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=App Name, master=local) created by __init__ at <ipython-input-2-4521e1ce2192>:1 

In [51]:
# read data from given file
def get_data(path):
    data_de = np.loadtxt(fname=path, dtype=float, usecols=(0,18), skiprows=2, delimiter=";")
    #data_bl = np.loadtxt(fname=data_path, dtype=str, usecols=(0,18), skiprows=2, delimiter=";")
        
    return data_de

In [52]:
def show_warming_stripes(data):
    ## stack data to be able to plot them with imshow
    temps = data
    # temps = temps_bremen
    # temps = temps_berlin_brandenburg

    stacked_temps = np.stack((temps, temps))

    vmin = 5.9
    vmax = 11
    ## plotting
    ###############
    plt.figure(figsize=(4,18))
    # img = plt.imshow(stacked_temps, cmap='RdBu_r', aspect=40, vmin=vmin, vmax=vmax)
    img = plt.imshow(stacked_temps, cmap='RdBu_r', aspect=40, )

    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0,
                hspace = 0, wspace = 0)
    plt.margins(0,0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())

In [76]:
months = []
data = []

for i in np.arange(start=1, stop=12):
    if i < 10:
        months.append("0"+str(i))
    else:
        months.append(str(i))
        
for month in months:
    data.append(list(get_data("../data/regional_averages_tm_"+month+".txt")))
print(data)

[[array([1881.  ,   -5.36]), array([1.882e+03, 4.100e-01]), array([ 1.883e+03, -3.200e-01]), array([1884.  ,    2.86]), array([1885.  ,   -2.79]), array([ 1.886e+03, -1.250e+00]), array([1887.  ,   -3.72]), array([1888.  ,   -1.93]), array([1889.  ,   -2.43]), array([1890.  ,    2.04]), array([1891.  ,   -4.54]), array([ 1.892e+03, -1.760e+00]), array([1893.  ,   -7.26]), array([ 1.894e+03, -1.830e+00]), array([1895.  ,   -4.03]), array([ 1.896e+03, -7.600e-01]), array([1897.  ,   -2.85]), array([1898.  ,    2.29]), array([1899.  ,    2.13]), array([1.9e+03, 7.1e-01]), array([1901.  ,   -3.73]), array([1902.  ,    2.76]), array([1.903e+03, 3.400e-01]), array([ 1.904e+03, -1.300e+00]), array([ 1.905e+03, -1.770e+00]), array([1.906e+03, 1.030e+00]), array([ 1.907e+03, -3.200e-01]), array([1908. ,   -2.2]), array([ 1.909e+03, -1.640e+00]), array([1.91e+03, 1.40e+00]), array([ 1.911e+03, -1.070e+00]), array([ 1.912e+03, -1.490e+00]), array([ 1.913e+03, -2.000e-01]), array([1914.  ,   -2.99

In [77]:
data_flattened = [pair for sublist in data for pair in sublist]
print(data_flattened)

[array([1881.  ,   -5.36]), array([1.882e+03, 4.100e-01]), array([ 1.883e+03, -3.200e-01]), array([1884.  ,    2.86]), array([1885.  ,   -2.79]), array([ 1.886e+03, -1.250e+00]), array([1887.  ,   -3.72]), array([1888.  ,   -1.93]), array([1889.  ,   -2.43]), array([1890.  ,    2.04]), array([1891.  ,   -4.54]), array([ 1.892e+03, -1.760e+00]), array([1893.  ,   -7.26]), array([ 1.894e+03, -1.830e+00]), array([1895.  ,   -4.03]), array([ 1.896e+03, -7.600e-01]), array([1897.  ,   -2.85]), array([1898.  ,    2.29]), array([1899.  ,    2.13]), array([1.9e+03, 7.1e-01]), array([1901.  ,   -3.73]), array([1902.  ,    2.76]), array([1.903e+03, 3.400e-01]), array([ 1.904e+03, -1.300e+00]), array([ 1.905e+03, -1.770e+00]), array([1.906e+03, 1.030e+00]), array([ 1.907e+03, -3.200e-01]), array([1908. ,   -2.2]), array([ 1.909e+03, -1.640e+00]), array([1.91e+03, 1.40e+00]), array([ 1.911e+03, -1.070e+00]), array([ 1.912e+03, -1.490e+00]), array([ 1.913e+03, -2.000e-01]), array([1914.  ,   -2.99]

In [86]:
rdd1 = sc.parallelize(data[1])
rdd1 = rdd1.map(lambda x: tuple(x))
print(rdd1.collect())

[(1881.0, -0.1), (1882.0, 1.54), (1883.0, 2.3), (1884.0, 2.66), (1885.0, 2.97), (1886.0, -2.89), (1887.0, -0.84), (1888.0, -2.56), (1889.0, -2.61), (1890.0, -2.58), (1891.0, -0.63), (1892.0, 0.53), (1893.0, 1.84), (1894.0, 1.6), (1895.0, -6.82), (1896.0, -0.23), (1897.0, 1.15), (1898.0, 1.08), (1899.0, 2.03), (1900.0, 1.13), (1901.0, -4.13), (1902.0, -1.25), (1903.0, 3.64), (1904.0, 1.1), (1905.0, 1.31), (1906.0, 0.34), (1907.0, -1.54), (1908.0, 1.21), (1909.0, -2.46), (1910.0, 2.44), (1911.0, 1.26), (1912.0, 2.27), (1913.0, 1.29), (1914.0, 3.05), (1915.0, 0.95), (1916.0, 0.83), (1917.0, -3.76), (1918.0, 1.68), (1919.0, -0.45), (1920.0, 2.97), (1921.0, 1.25), (1922.0, -1.26), (1923.0, 0.98), (1924.0, -2.63), (1925.0, 3.64), (1926.0, 4.39), (1927.0, 0.86), (1928.0, 2.5), (1929.0, -9.6), (1930.0, -0.19), (1931.0, -1.16), (1932.0, -1.89), (1933.0, 0.12), (1934.0, 1.17), (1935.0, 1.89), (1936.0, 0.23), (1937.0, 2.23), (1938.0, 0.93), (1939.0, 1.63), (1940.0, -4.31), (1941.0, -0.22), (1942.

In [96]:
rdd2 = sc.parallelize(data_flattened)
rdd2 = rdd2.map(lambda x: tuple(x))
print(rdd2.collect())

[(1881.0, -5.36), (1882.0, 0.41), (1883.0, -0.32), (1884.0, 2.86), (1885.0, -2.79), (1886.0, -1.25), (1887.0, -3.72), (1888.0, -1.93), (1889.0, -2.43), (1890.0, 2.04), (1891.0, -4.54), (1892.0, -1.76), (1893.0, -7.26), (1894.0, -1.83), (1895.0, -4.03), (1896.0, -0.76), (1897.0, -2.85), (1898.0, 2.29), (1899.0, 2.13), (1900.0, 0.71), (1901.0, -3.73), (1902.0, 2.76), (1903.0, 0.34), (1904.0, -1.3), (1905.0, -1.77), (1906.0, 1.03), (1907.0, -0.32), (1908.0, -2.2), (1909.0, -1.64), (1910.0, 1.4), (1911.0, -1.07), (1912.0, -1.49), (1913.0, -0.2), (1914.0, -2.99), (1915.0, 0.2), (1916.0, 3.76), (1917.0, -2.59), (1918.0, 0.69), (1919.0, 0.67), (1920.0, 2.3), (1921.0, 4.2), (1922.0, -2.68), (1923.0, 1.51), (1924.0, -2.89), (1925.0, 2.49), (1926.0, -0.02), (1927.0, 1.77), (1928.0, 1.57), (1929.0, -4.69), (1930.0, 2.28), (1931.0, 0.18), (1932.0, 1.82), (1933.0, -2.6), (1934.0, 0.45), (1935.0, -0.76), (1936.0, 3.05), (1937.0, -0.84), (1938.0, 1.69), (1939.0, 2.55), (1940.0, -9.0), (1941.0, -5.37)

In [106]:
result_local = rdd2.reduceByKeyLocally()
print(result)

{1881.0: 87.73, 1882.0: 100.07000000000001, 1883.0: 94.6, 1884.0: 102.78, 1885.0: 92.92000000000002, 1886.0: 96.18, 1887.0: 83.41, 1888.0: 82.25, 1889.0: 88.60999999999999, 1890.0: 87.69, 1891.0: 89.16, 1892.0: 89.98, 1893.0: 94.89, 1894.0: 97.53999999999999, 1895.0: 87.72, 1896.0: 90.94999999999999, 1897.0: 95.35000000000001, 1898.0: 102.05, 1899.0: 97.04999999999998, 1900.0: 100.2, 1901.0: 91.71000000000001, 1902.0: 86.07000000000002, 1903.0: 100.23, 1904.0: 100.26, 1905.0: 95.44999999999999, 1906.0: 99.45, 1907.0: 93.62000000000002, 1908.0: 89.68, 1909.0: 88.21000000000001, 1910.0: 100.89000000000001, 1911.0: 108.57, 1912.0: 94.28, 1913.0: 102.34999999999998, 1914.0: 102.22999999999999, 1915.0: 94.96000000000001, 1916.0: 101.38, 1917.0: 90.45, 1918.0: 102.4, 1919.0: 87.80999999999999, 1920.0: 103.53000000000002, 1921.0: 107.99, 1922.0: 86.29999999999998, 1923.0: 95.85999999999999, 1924.0: 90.33, 1925.0: 99.83, 1926.0: 104.88000000000001, 1927.0: 96.42999999999999, 1928.0: 100.15, 19

In [ ]:
result = rdd2.reduceByKey(lambda a, b: a + b)
print(result.collect())

In [74]:
text_file = sc.textFile("../data/regional_averages_tm_01.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("result.txt")



In [75]:
print(counts.collect())

[('Zeitreihen', 1), ('fuer', 2), ('Gebietsmittel', 1), ('Bundeslaender', 1), ('und', 1), ('Kombinationen', 1), ('von', 1), ('Bundeslaender,', 1), ('erstellt', 1), ('am:', 1), ('20201104', 1), ('Jahr;Monat;Brandenburg/Berlin;Brandenburg;Baden-Wuerttemberg;Bayern;Hessen;Mecklenburg-Vorpommern;Niedersachsen;Niedersachsen/Hamburg/Bremen;Nordrhein-Westfalen;Rheinland-Pfalz;Schleswig-Holstein;Saarland;Sachsen;Sachsen-Anhalt;Thueringen/Sachsen-Anhalt;Thueringen;Deutschland;', 1), ('1881;01;', 1), ('', 8392), ('-5.54;', 1), ('-5.56;', 2), ('-4.89;', 2), ('-6.51;', 1), ('-5.68;', 2), ('-5.07;', 2), ('-4.55;', 2), ('-4.21;', 3), ('-4.49;', 1), ('-4.06;', 4), ('-4.15;', 2), ('-6.22;', 1), ('-5.89;', 1), ('-6.28;', 2), ('-6.76;', 1), ('-5.36;', 2), ('1882;01;', 1), ('1.43;', 4), ('1.42;', 2), ('-0.65;', 2), ('-1.33;', 5), ('-0.30;', 5), ('1.59;', 4), ('1.73;', 7), ('1.26;', 4), ('-0.17;', 5), ('2.11;', 2), ('-0.04;', 3), ('0.63;', 6), ('1.18;', 6), ('0.60;', 3), ('-0.12;', 5), ('0.41;', 5), ('1883

SyntaxError: invalid syntax (<ipython-input-94-f1f97ed4ffee>, line 1)